## Overall Prediction Observation


In [1]:
# Define variables
IS_FILTERED = True
ARCHIVED = None

SHOW_HEADER = True
SHOW_LAST_APPEARANCE = None

In [2]:
# Import libraries
import sys
sys.path.append('../../../prediction')

import numpy as np
import matplotlib.pyplot as plt

from putils.observation import compute_rmse, compute_mae, compute_mape, load_data_from_tuned_folder

In [3]:
# Load data
dfs = {
    "l1": load_data_from_tuned_folder("l1", last_appearance=SHOW_LAST_APPEARANCE),
    "l2": load_data_from_tuned_folder("l2", last_appearance=SHOW_LAST_APPEARANCE),
    "l3": load_data_from_tuned_folder("l3", last_appearance=SHOW_LAST_APPEARANCE),
}

# Make L1 and L2 data have the same length with L3 if SHOW_LAST_APPEARANCE = None
if SHOW_LAST_APPEARANCE is None:
    dfs["l1"] = dfs["l1"].tail(len(dfs["l3"]))
    dfs["l2"] = dfs["l2"].tail(len(dfs["l3"]))

# Indicate which columns to ignore
IGNORED_COLS = ["Time", "Actual"] # No need to include "FormattedTime" because it is used as index
if IS_FILTERED: IGNORED_COLS.append("Raw")

In [4]:
# Compute RMSE, MAE, and MAPE for each model
for key, df in dfs.items():
    print("Layer", key)
    for column in df.columns:
        if column not in IGNORED_COLS:
            actual = df["Raw"] if IS_FILTERED else df["Actual"]
            rmse = compute_rmse(actual, df[column])
            mape = compute_mape(actual, df[column])
            mae = compute_mae(actual, df[column])
            print(f"{column} - RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%")
    print("------------------")

Layer l1
ARIMA - RMSE: 0.0228, MAE: 0.0153, MAPE: 2.80%
RNN - RMSE: 0.0149, MAE: 0.0103, MAPE: 1.88%
LSTM - RMSE: 0.0155, MAE: 0.0106, MAPE: 1.94%
CNN - RMSE: 0.0159, MAE: 0.0108, MAPE: 1.97%
GRU - RMSE: 0.0152, MAE: 0.0105, MAPE: 1.92%
------------------
Layer l2
LINEAR_REGRESSION - RMSE: 0.0149, MAE: 0.0103, MAPE: 1.87%
RANDOM_FOREST - RMSE: 0.0153, MAE: 0.0105, MAPE: 1.90%
FEEDFORWARD_NEURAL_NETWORK - RMSE: 0.0149, MAE: 0.0103, MAPE: 1.87%
------------------
Layer l3
Predicted - RMSE: 0.0149, MAE: 0.0103, MAPE: 1.87%
------------------


In [5]:
# Get data from d1
d1 = dfs["l1"]
d1 = d1.drop(columns=["Time", "Actual"])
alpha = 100

# Dict of models with their initial weights (1 / number of models)
weights = {}
for column in d1.columns:
    if column != "Average" and column != "Raw":
        weights[column] = 1 / (len(d1.columns) - 1) # -1 because the column "Raw" is ignored

# Create a new column for the average value
d1["Average"] = None

# Loop through each row and compute the weighted average
count = 0
for index, row in d1.iterrows():
    # Find the average value with weight of the row 
    total = 0
    for column in d1.columns:
        if column != "Average" and column != "Raw":
            total += row[column] * weights[column]
    # Add the average value to the row
    d1.loc[index, "Average"] = total
    # Compute RMSE of each model from 0 to count - 1
    rmses = {}
    for column in d1.columns:
        if column != "Average" and column != "Raw":
            rmses[column] = compute_rmse(d1["Raw"][:count], d1[column][:count])
    # Compute the sum of exponential RMSEs [exp(alpha * -rmse)]
    rmse_sum = sum([np.exp(alpha * -rmse) for rmse in rmses.values()])
    # Update the weights
    for column in d1.columns:
        if column != "Average" and column != "Raw":
            result = np.exp(alpha * -rmses[column]) / rmse_sum
            # Check if nan
            if np.isnan(result) == False:
                weights[column] = result
    # Increment count
    count += 1

print(f"final weights: {weights}")

# Save the data into new file
d1.to_csv("l1_weighted_average.csv")

final weights: {'ARIMA': 0.10612829763874386, 'RNN': 0.23460247819115762, 'LSTM': 0.21951357191482349, 'CNN': 0.21231286556396534, 'GRU': 0.22744278669130955}
